# Diabetic retinopathy analysis
Simo Hyttinen<br>
Last edited: 25.02.2018<br>
Cognitive Systems for Health Technology Applications<br>
<a href="https://www.metropolia.fi/en">Helsinki Metropolia University of Applied Sciences</a>


## 1. Objectives

The point of this assignment is to import medical image data and to create a convolutional neural network to detect diabetic retinopathy.

## 2. Required libraries

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import keras
import imageio as im
from keras.preprocessing.image import ImageDataGenerator
import time

Using TensorFlow backend.


In [3]:
# This cell is to be run only when the dataset is not in local storage
from urllib.request import urlretrieve
url = r"https://github.com/Nomikxyz/retinopathy-dataset/archive/master.zip"
dst = 'master.zip'
urlretrieve(url, dst)
from zipfile import ZipFile
zip_ref = ZipFile('master.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [4]:
# This cell shows you if the dataset is in local storage
import os
os.listdir('retinopathy-dataset-master')

['nosymptoms', 'README.md', 'symptoms']

I manually separated 150 images to be a validation data set, 100 from nosymptoms and 50 from symptoms folders, into a folder called retinopathy-validation/

## 3. Data description and preprocessing

In [5]:
img_width, img_height = 512, 512
input_width, input_height = 256, 256
train_data_dir = 'retinopathy-dataset-master/'
val_data_dir = 'retinopathy-validation/'

## 4. Modeling and compilation

In [6]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3, 3), input_shape=(input_width, input_height, 3), activation="relu"))
model.add(keras.layers.Conv2D(16, (3, 3), activation="relu"))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Dropout(0.15))
model.add(keras.layers.Conv2D(32, (3, 3), activation="relu"))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation="relu"))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Dropout(0.15))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(1, activation = "sigmoid"))


model.compile(loss = 'binary_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 16)      2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 16)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 126, 126, 16)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 124, 32)      4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 60, 60, 32)        9248      
__________

## 5. Training and Validation


In [10]:
epochs = 15
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(input_width, input_height),
        batch_size=batch_size,
        class_mode='binary')

val_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(input_width, input_height),
        batch_size=batch_size,
        class_mode='binary')

t1 = time.time()
history = model.fit_generator(
        train_generator,
        steps_per_epoch = 1913 // batch_size + 1,
        validation_data=val_generator,
        validation_steps=150 // batch_size + 1,
        epochs = epochs)

t2 = time.time()
model.save('Case2model.h5')

Found 1913 images belonging to 2 classes.
Found 150 images belonging to 2 classes.
Epoch 1/15
120/120 [==============================] - 519s 4s/step - loss: 0.5525 - acc: 0.7222 - val_loss: 0.7322 - val_acc: 0.4200
Epoch 2/15
120/120 [==============================] - 515s 4s/step - loss: 0.4990 - acc: 0.7647 - val_loss: 0.7571 - val_acc: 0.4933
Epoch 3/15
120/120 [==============================] - 515s 4s/step - loss: 0.4758 - acc: 0.7745 - val_loss: 0.9420 - val_acc: 0.4267
Epoch 4/15
120/120 [==============================] - 510s 4s/step - loss: 0.4423 - acc: 0.8047 - val_loss: 1.0368 - val_acc: 0.5133
Epoch 5/15
120/120 [==============================] - 519s 4s/step - loss: 0.4429 - acc: 0.7970 - val_loss: 0.7591 - val_acc: 0.4933
Epoch 6/15
120/120 [==============================] - 540s 4s/step - loss: 0.4288 - acc: 0.8014 - val_loss: 0.7640 - val_acc: 0.4933
Epoch 7/15
120/120 [==============================] - 526s 4s/step - loss: 0.4282 - acc: 0.8064 - val_loss: 0.7728 - va

I ran this on the IBM DSE server to cut back on time spent training the network. I left it to run while I went to work and when I came back the notebook kernel had stopped and I hadn't saved the model, so I restarted the kernel and reran the code and made sure to save the model and tried for a couple of hours to find out how to download the file to my PC or even send it via email or FTP but nothing I tried worked. When I got back to work on it in the morning, the files were wiped off the server. <br><br>
Later I just did it on my own PC, which saved me a lot of trouble and wasn't significantly slower. 

## 6. Evaluation

## 7. Results and Discussion

## 8. Conclusions